In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import math
from shapely.ops import cascaded_union
from sklearn.cluster import DBSCAN
import mplleaflet
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn import metrics
from matplotlib.colors import LinearSegmentedColormap

In [2]:
def load_data():
    #Loads the CSV files and appends them into a single DataFrame
    column_types = {'Accident_Index': np.string_, 'LSOA_of_Accident_Location': np.string_}
    data13 = pd.read_csv('/home/shreyas/Desktop/major2/data/DfTRoadSafety_Accidents_2013.csv', dtype=column_types)
    data14 = pd.read_csv('/home/shreyas/Desktop/major2/data/DfTRoadSafety_Accidents_2014.csv', dtype=column_types)
    data15 = pd.read_csv('/home/shreyas/Desktop/major2/data/DfTRoadSafety_Accidents_2015.csv', dtype=column_types)
    data16 = pd.read_csv('/home/shreyas/Desktop/major2/data/DftRoadSafety_Accidents_2016.csv', dtype=column_types)
    return data16.append(data15.append(data14.append(data13)))

In [3]:
data = load_data()

In [4]:
data.shape

(561689, 32)

In [5]:
data=data[pd.to_numeric(data['Latitude'], errors='coerce').notnull()]  
data=data[pd.to_numeric(data['Longitude'], errors='coerce').notnull()] 
data.Latitude.astype('float64')
data.Longitude.astype('float64')
data.describe()


,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Day_of_Week,Local_Authority_(District),...,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident
count,561654.000000,5.616540e+05,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,...,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000,561654.000000
mean,446806.880880,2.905954e+05,-1.332075,52.502879,30.060094,2.830483,1.837024,1.327670,4.105846,345.172770,...,358.135359,0.008493,0.836978,1.953160,1.526707,1.316654,0.095529,0.064883,1.347249,1.203889
std,95247.243189,1.584065e+05,1.398938,1.426531,25.278252,0.405227,0.711631,0.811918,1.914869,258.773383,...,1256.923309,0.127801,1.930658,1.654816,1.575345,0.584271,0.686510,0.594162,0.476242,0.412539
min,66435.000000,1.029000e+04,-7.489410,49.912941,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,384599.250000,1.765452e+05,-2.231652,51.474671,6.000000,3.000000,1.000000,1.000000,2.000000,106.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
50%,451293.500000,2.417900e+05,-1.239750,52.057233,30.000000,3.000000,2.000000,1.000000,4.000000,324.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
75%,526790.000000,3.912680e+05,-0.171623,53.415400,45.000000,3.000000,2.000000,1.000000,6.000000,517.000000,...,0.000000,0.000000,0.000000,4.000000,1.000000,2.000000,0.000000,0.000000,2.000000,1.000000
max,655309.000000,1.197917e+06,1.758797,60.661117,98.000000,3.000000,67.000000,93.000000,7.000000,941.000000,...,9999.000000,2.000000,8.000000,7.000000,9.000000,5.000000,7.000000,7.000000,2.000000,3.000000


In [6]:
# Create the radian longitude and latitude columns


data['rad_lng'] = data['Longitude'] * math.pi / 180.0
data['rad_lat'] = data['Latitude'] * math.pi / 180.0

In [7]:
eps_in_meters = 50.0
num_samples = 10
earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)


In [8]:
data['cluster'] = DBSCAN(eps=eps_in_radians, min_samples=num_samples, metric='haversine').fit_predict(data[['rad_lat', 'rad_lng']])

In [9]:
labels= data['cluster'].to_frame()['cluster'].to_numpy()   #Dataframe to numpy array
print(labels)
print(len(labels))
type(labels)

[ 0 -1 -1 ... -1 -1 -1]
561654


numpy.ndarray

In [12]:
len(set(labels))

3833

In [13]:
labels_true=data['Accident_Severity'].to_frame()['Accident_Severity'].to_numpy()   #Dataframe to numpy array
print(len(labels_true))
print(labels_true)

561654
[3 3 3 ... 3 2 3]


In [14]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))

Homogeneity: 0.017
Completeness: 0.006
V-measure: 0.009
